In [88]:
from pathlib import Path
from tqdm import tqdm
import numpy as np

In [33]:
DATA_TXT_DIR='TextANIMAR2023/'
DATA_SKT_DIR='SketchANIMAR2023/'

In [34]:
!find $DATA_TXT_DIR -maxdepth 2

TextANIMAR2023/
TextANIMAR2023/3D_Model_References
TextANIMAR2023/3D_Model_References/generated_models
TextANIMAR2023/3D_Model_References/References.csv
TextANIMAR2023/3D_Model_References/References.zip
TextANIMAR2023/3D_Model_References/References
TextANIMAR2023/Train
TextANIMAR2023/Train/TextQuery_Train.csv
TextANIMAR2023/Train/TextQuery_GT_Train.csv


In [7]:
!./blender-2.79/blender -b -P generate_ring.py -- $DATA_DIR/3D_Model_References/References
# blender-2.79/blender -b -P generate_ring.py -- SketchANIMAR2023/3D_Model_References/References

found bundled python: /home/nhtlong/shrec/baseline/SHREC23-ANIMAR-BASELINE/data/blender-2.79/2.79/python
/home/nhtlong/.shrec/save.txt
Done

Blender quit


In [ ]:
!./blender-2.79/blender -b -P generate_ring.py -- $DATA_DIR/3D_Model_References/References
# blender-2.79/blender -b -P generate_ring.py -- SketchANIMAR2023/3D_Model_References/References

In [35]:
!find $DATA_TXT_DIR -maxdepth 2

TextANIMAR2023/
TextANIMAR2023/3D_Model_References
TextANIMAR2023/3D_Model_References/generated_models
TextANIMAR2023/3D_Model_References/References.csv
TextANIMAR2023/3D_Model_References/References.zip
TextANIMAR2023/3D_Model_References/References
TextANIMAR2023/Train
TextANIMAR2023/Train/TextQuery_Train.csv
TextANIMAR2023/Train/TextQuery_GT_Train.csv


In [36]:
DATA_TXT_DIR = Path(DATA_TXT_DIR)
DATA_SKT_DIR = Path(DATA_SKT_DIR)

In [37]:
import pandas as pd

In [61]:
model_paths = list((DATA_SKT_DIR/'3D_Model_References'/'References').glob('*.obj'))
available_models = [i.stem for i in model_paths]

model_sketch_paths = list((DATA_SKT_DIR/'Train'/'SketchQuery_Train').glob('*.jpg'))
sketch_queries_gt = pd.read_csv(DATA_SKT_DIR/'Train'/'SketchQuery_GT_Train.csv',delimiter=';')
sketch_queries_gt.rename(columns={'Sketch Query ID': 'ID'}, inplace=True)
available_sketches = pd.read_csv(DATA_SKT_DIR/'Train'/'SketchQuery_Train.csv',delimiter=';').ID.values

text_queries_gt = pd.read_csv(DATA_TXT_DIR/'Train'/'TextQuery_GT_Train.csv',delimiter=';')
text_queries_gt.rename(columns={'Text Query ID': 'ID'}, inplace=True)
available_text = pd.read_csv(DATA_TXT_DIR/'Train'/'TextQuery_Train.csv',delimiter=';').set_index('ID').to_dict()['Description']

In [66]:
def check_mapping(query_df, available_models, available_queries):
    for i, row in tqdm(query_df.iterrows(), total=len(query_df)):
        query = row['ID']
        model = row['Model ID']
        assert query in available_queries, f'Query {query} is not available'
        assert model in available_models, f'Model {model} is not available'

check_mapping(sketch_queries_gt, available_models, available_sketches)
check_mapping(text_queries_gt, available_models, available_text.keys())

100%|██████████| 382/382 [00:00<00:00, 20474.40it/s]


In [92]:
def split(df, unique_col = 'Sketch Query ID', split=0.8):
    # split by unique sket_filename to avoid data leakage
    sket_filenames = df[unique_col].unique()
    np.random.shuffle(sket_filenames)
    train_sket_filenames = sket_filenames[:int(len(sket_filenames)*split)]
    val_sket_filenames = sket_filenames[int(len(sket_filenames)*split):]
    train_df = df[df[unique_col].isin(train_sket_filenames)]
    val_df = df[df[unique_col].isin(val_sket_filenames)]
    return train_df, val_df

In [93]:
data_skt_df = pd.DataFrame(columns=['obj_filename', 'sket_filename'])
data_skt_df['obj_filename'] = sketch_queries_gt['Model ID'].apply(lambda x: f'{x}.obj')
data_skt_df['sket_filename'] = sketch_queries_gt['ID'].apply(lambda x: f'{x}.jpg')
print(len(data_skt_df))
data_skt_df.head()

296


obj_filename         sket_filename
0  90ba6644ad322c47.obj  036dbe8e06efc868.jpg
1  e9b85e6177ccf14d.obj  036dbe8e06efc868.jpg
2  2d0e5e5ef940e212.obj  036dbe8e06efc868.jpg
3  55a985eef253a7a0.obj  036dbe8e06efc868.jpg
4  90ba6644ad322c47.obj  0516d1f786008dce.jpg

In [102]:
train_skt_df, test_skt_df = split(data_skt_df, unique_col = 'sket_filename', split=0.8)
train_skt_df.to_csv('csv/train_skt.csv', index=False)
test_skt_df.to_csv('csv/test_skt.csv', index=False)
len(train_skt_df), len(test_skt_df)

(236, 60)

In [108]:
data_txt_df = pd.DataFrame(columns=['obj_filename', 'tex'])
data_txt_df['obj_filename'] = text_queries_gt['Model ID'].apply(lambda x: f'{x}.obj')
data_txt_df['tex'] = text_queries_gt['ID'].apply(lambda x: available_text[x])
print(len(data_txt_df))
data_txt_df.head()

382


obj_filename                                                tex
0  13a49ab0aecbc76f.obj  A female mandrill is climbing out the top of t...
1  40e86f26ce8cc8eb.obj  A female mandrill is climbing out the top of t...
2  836406771c68c193.obj  A female mandrill is climbing out the top of t...
3  bda1f6d4a4b5fe79.obj  A female mandrill is climbing out the top of t...
4  70c98f4c562affee.obj              A black goat is returning the shelter

In [113]:
train_txt_df, test_txt_df = split(data_txt_df, unique_col = 'tex', split=0.8)
train_txt_df.to_csv('csv/train_tex.csv', index=False)
test_txt_df.to_csv('csv/test_tex.csv', index=False)
len(train_txt_df), len(test_txt_df)

(302, 80)

In [116]:
# merge the two datasets by obj_filename
train_df = pd.merge(data_skt_df, data_txt_df, on='obj_filename')
print(len(train_df))
train_df.head()

240


obj_filename         sket_filename  \
0  90ba6644ad322c47.obj  036dbe8e06efc868.jpg   
1  90ba6644ad322c47.obj  0516d1f786008dce.jpg   
2  90ba6644ad322c47.obj  1f4391cd4503420a.jpg   
3  90ba6644ad322c47.obj  86889d264071b08d.jpg   
4  e9b85e6177ccf14d.obj  036dbe8e06efc868.jpg   

                                                 tex  
0  A male African elephant is drinking and bathin...  
1  A male African elephant is drinking and bathin...  
2  A male African elephant is drinking and bathin...  
3  A male African elephant is drinking and bathin...  
4  A male African elephant is drinking and bathin...

In [117]:
train_obj_df, test_obj_df = split(train_df, unique_col = 'obj_filename', split=0.8)
train_obj_df.to_csv('csv/train_both.csv', index=False)
test_obj_df.to_csv('csv/test_both.csv', index=False)
len(train_obj_df), len(test_obj_df)

(185, 55)